In [1]:
# Установка необходимых зависимостей
!pip install ultralytics
!pip install torch torchvision
!pip install albumentations

# Настройка CUDA для лучшего управления памятью
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Клонирование репозитория с обученной моделью
!git clone https://github.com/smicurin474/Car_plate_detecting.git

# Добавляем путь к репозиторию в PYTHONPATH
import sys
sys.path.append('/kaggle/working/Car_plate_detecting')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 17.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninst

In [2]:
# Загрузка датасета
!mkdir -p /kaggle/working/finetune_dataset
!wget https://huggingface.co/datasets/AY000554/Car_plate_detecting_dataset/resolve/main/train.zip
!wget https://huggingface.co/datasets/AY000554/Car_plate_detecting_dataset/resolve/main/val.zip

# Распаковка датасета
!unzip -q train.zip -d /kaggle/working/finetune_dataset
!unzip -q val.zip -d /kaggle/working/finetune_dataset

--2025-09-03 15:52:22--  https://huggingface.co/datasets/AY000554/Car_plate_detecting_dataset/resolve/main/train.zip
Resolving huggingface.co (huggingface.co)... 18.244.202.68, 18.244.202.60, 18.244.202.73, ...
Connecting to huggingface.co (huggingface.co)|18.244.202.68|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/f5/fb/f5fb9d95f122eb06ad36fa568b80b423cda743737da1285f498b17999aee4770/689e09a59fd1ba57584e45896ae9d87a205a201080d23116362d41c5e1b805c3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27train.zip%3B+filename%3D%22train.zip%22%3B&response-content-type=application%2Fzip&Expires=1756918342&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1NjkxODM0Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2Y1L2ZiL2Y1ZmI5ZDk1ZjEyMmViMDZhZDM2ZmE1NjhiODBiNDIzY2RhNzQzNzM3ZGExMjg1ZjQ5OGIxNzk5OWFlZTQ3NzAvNjg5ZTA5YTU5ZmQxYmE1NzU4NGU0NTg5NmFlOWQ4N2EyMDVhMjAxMDgwZD

In [4]:
# Создаем отдельную директорию для тестирования аугментаций на сложных случаях
!mkdir -p /kaggle/working/hard_cases_test/images
!cp -r /kaggle/working/Car_plate_detecting/hard_cases/* /kaggle/working/hard_cases_test/images/

print("Количество изображений для дообучения:", len(os.listdir('/kaggle/working/finetune_dataset/train/images')))
print("Количество изображений для валидации:", len(os.listdir('/kaggle/working/finetune_dataset/val/images')))
print("Количество сложных случаев для тестирования:", len(os.listdir('/kaggle/working/hard_cases_test/images')))

Количество изображений для дообучения: 20505
Количество изображений для валидации: 2563
Количество сложных случаев для тестирования: 37


In [5]:
# Настройка конфигурации для дообучения
import configparser
from pathlib import Path

# Загрузка базовой конфигурации
config = configparser.ConfigParser()
config.read("/kaggle/working/Car_plate_detecting/Config/Config.ini")

# Модификация настроек для дообучения
train_config = config["Train"]

# Путь к предобученной модели
train_config["weights"] = "/kaggle/input/plate_detection/pytorch/default/1"

# Пути к датасету
train_config["train"] = "/kaggle/working/finetune_dataset/train"
train_config["val"] = "/kaggle/working/finetune_dataset/val"

# Настройка аугментаций для решения выявленных проблем
train_config["degrees"] = "10.0"     # Небольшой поворот
train_config["scale"] = "0.5"        # Масштабирование
train_config["perspective"] = "0.001" # Легкая перспектива
train_config["flipud"] = "0.3"       # Разрешаем вертикальный переворот
train_config["fliplr"] = "0.3"       # Разрешаем горизонтальный переворот
train_config["mosaic"] = "1.0"       # Включаем мозаику
train_config["mixup"] = "0.3"        # Включаем mixup

# Параметры для быстрого дообучения
train_config["epochs"] = "15"        # Немного увеличиваем эпохи для лучшей адаптации
train_config["batch_size"] = "32"    # Сохраняем большой batch size
train_config["patience"] = "7"       # Увеличиваем patience для лучшей сходимости
train_config["optimizer"] = "AdamW"  # Используем AdamW для дообучения
train_config["lr0"] = "0.0001"      # Небольшой learning rate для тонкой настройки

# Сохраняем обновленную конфигурацию
with open('/kaggle/working/Car_plate_detecting/Config/Config_finetune.ini', 'w') as configfile:
    config.write(configfile)

In [ ]:
# Создаем YAML конфигурацию для YOLO
import yaml

# Создаем конфигурацию в формате YAML
yolo_config = {
    'path': '/kaggle/working/finetune_dataset',  # Путь к датасету
    'train': 'train/images',  # Путь к тренировочным изображениям
    'val': 'val/images',      # Путь к валидационным изображениям
    
    'names': {
        0: 'plate'  # Название класса
    },
    
    # Настройки обучения
    'epochs': int(train_config["epochs"]),
    'batch': int(train_config["batch_size"]),
    'optimizer': train_config["optimizer"],
    'lr0': float(train_config["lr0"]),
    'patience': int(train_config["patience"]),
    
    # Аугментации
    'degrees': float(train_config["degrees"]),
    'scale': float(train_config["scale"]),
    'perspective': float(train_config["perspective"]),
    'flipud': float(train_config["flipud"]),
    'fliplr': float(train_config["fliplr"]),
    'mosaic': float(train_config["mosaic"]),
    'mixup': float(train_config["mixup"]),
    
    # Дополнительные аугментации для сложных случаев
    'hsv_h': 0.015,  # Изменение оттенка
    'hsv_s': 0.7,    # Насыщенность
    'hsv_v': 0.4,    # Яркость
    'blur': 0.3,     # Размытие
    'contrast': 0.3, # Контраст
}

# Сохраняем YAML конфигурацию
yaml_path = '/kaggle/working/Car_plate_detecting/Config/finetune.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(yolo_config, f, sort_keys=False)

# Загружаем модель
model = YOLO("/kaggle/input/plate_detection/pytorch/default/1/best.pt")

# История детекций на сложных случаях
hard_cases_history = []

def on_epoch_end(epoch):
    detection_rate = validate_on_hard_cases(
        model, 
        '/kaggle/working/hard_cases_test/images', 
        epoch
    )
    hard_cases_history.append(detection_rate)
    
    plt.figure(figsize=(10, 5))
    plt.plot(hard_cases_history, marker='o')
    plt.title('Прогресс на сложных случаях')
    plt.xlabel('Эпоха')
    plt.ylabel('Процент детекций')
    plt.grid(True)
    plt.savefig('hard_cases_progress.png')
    plt.close()

# Запускаем обучение
model.train(
    data=yaml_path,  # Используем YAML конфигурацию
    epochs=int(train_config["epochs"]),
    resume=False
)

# Проверяем на сложных случаях после обучения
final_detection_rate = validate_on_hard_cases(
    model, 
    '/kaggle/working/hard_cases_test/images', 
    int(train_config["epochs"])
)

Ultralytics 8.3.192 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/Car_plate_detecting/Config/finetune.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/input/plate_detection/pytorch/default/1/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_ma